In [24]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow

### download and clean data

In [4]:
con_data = pd.read_csv('https://cocl.us/concrete_data')
con_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
con_data.shape

(1030, 9)

In [7]:
con_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [8]:
con_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### data cleaned

In [10]:
con_data_column=con_data.columns
X = con_data[con_data_column[con_data_column!= 'Strength']]
y = con_data['Strength']
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [11]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### inputs of NN

In [13]:
n_cols= X.shape[1]

# Part A: Build a baseline model

### A.1: Split the data

In [15]:
from sklearn.model_selection import train_test_split
# split after, in train_and_test function

Train set: (721, 8) (721,)
Test set: (309, 8) (309,)


### A.2: The regression model

In [39]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

def regression_model(network_arch, n_cols, optimizer='adam', loss='mean_squared_error'):
    '''
    network_arch: 
          {"layer_1": {"n_nodes": n1, "activation": activation1},
           ...
           "layer_n": {"n_nodes: nn}
          }
    '''
    model = Sequential()
    n_layers = len(network_arch)
    output_layer = "layer_"+str(n_layers)
    
    for i,layer in network_arch.items():
        if i == "layer_1":
            model.add(Dense(int(layer.get("n_nodes")), activation=layer.get("activation"), input_shape=(n_cols,)))
        elif i == output_layer:
            model.add(Dense(int(layer.get("n_nodes"))))
        else:
            model.add(Dense(int(layer.get("n_nodes")), activation=layer.get("activation")))
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [46]:
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

def train_and_test(X, y, network_arch, n_cols, optimizer , loss, n_times, epochs, norm, seed):
    '''
    seed: for spilt data
    norm: (bool) normalization or not
    '''
    mse_list=[]
    for i in range(n_times):
        if norm:
            X= preprocessing.StandardScaler().fit(X).transform(X.astupe(float))
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed+i)
        model = regression_model(network_arch=network_arch, n_cols=n_cols)
        model.fit(X_train, y_train, epochs= epochs, verbose=2)
        y_hat= model.predict(X_test)
        mse = mean_squared_error(y_test, y_hat)
        print("time {} : mse= {}".format(i+1,mse))
        mse_list.append(mse)
    return mse_list

### A.3: Part A result


In [41]:
model_archA = {"layer_1": {"n_nodes": 10, "activation": "relu"}, "layer_2": {"n_nodes": 1},
                }

In [47]:
mse_listA=train_and_test(X=X, y=y, network_arch=model_archA, n_cols=n_cols, optimizer="adam", loss="mean_squared_error", 
                        n_times=50, epochs=50, norm= False, seed=360)

Epoch 1/50
23/23 - 1s - loss: 555197.3125 - 655ms/epoch - 28ms/step
Epoch 2/50
23/23 - 0s - loss: 305396.2188 - 40ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 153635.5000 - 40ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 64926.4414 - 49ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 20582.6523 - 46ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 5621.1050 - 40ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 3146.6445 - 39ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 2918.8630 - 40ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 2806.9006 - 45ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 2699.3901 - 48ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 2597.1755 - 43ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 2493.9045 - 48ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 2398.1772 - 43ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 2300.5593 - 45ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 2210.3655 - 40ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s -

Epoch 27/50
23/23 - 0s - loss: 1048.6646 - 44ms/epoch - 2ms/step
Epoch 28/50
23/23 - 0s - loss: 1023.5817 - 46ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 1000.6996 - 42ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 976.8897 - 36ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 954.4331 - 37ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 929.3522 - 41ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 906.9787 - 37ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 885.8734 - 40ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 863.3935 - 38ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 843.0884 - 35ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 820.9233 - 34ms/epoch - 1ms/step
Epoch 38/50
23/23 - 0s - loss: 802.5897 - 36ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 782.1384 - 36ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 764.6769 - 37ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 746.8289 - 37ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 728.05

Epoch 4/50
23/23 - 0s - loss: 4425.1724 - 45ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 3480.1426 - 47ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 2748.9519 - 47ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 2163.9968 - 78ms/epoch - 3ms/step
Epoch 8/50
23/23 - 0s - loss: 1742.5771 - 48ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 1341.3297 - 36ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1054.9133 - 36ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 874.9839 - 40ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 746.3344 - 37ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 651.0549 - 39ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 556.2595 - 37ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 495.3682 - 35ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 439.2293 - 42ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 397.7136 - 36ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 367.1653 - 35ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 341.4731

Epoch 31/50
23/23 - 0s - loss: 707.5231 - 46ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 636.5955 - 44ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 573.8098 - 43ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 522.9389 - 39ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 474.6313 - 37ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 435.2224 - 37ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 409.6638 - 40ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 380.7542 - 37ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 346.4145 - 38ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 326.3218 - 38ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 307.2707 - 40ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 294.0612 - 36ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 280.8785 - 35ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 267.6604 - 35ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 259.1096 - 35ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 252.7129 

Epoch 8/50
23/23 - 0s - loss: 4842.7397 - 52ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 4214.2217 - 48ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 3708.3220 - 44ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 3213.0544 - 45ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 2848.0662 - 39ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 2519.2332 - 38ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 2214.5715 - 41ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 1977.1798 - 46ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 1745.7723 - 42ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 1553.2050 - 41ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 1409.4102 - 39ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 1268.0240 - 40ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 1148.0385 - 42ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 1007.8725 - 40ms/epoch - 2ms/step
Epoch 22/50
23/23 - 0s - loss: 915.0838 - 40ms/epoch - 2ms/step
Epoch 23/50
23/23 - 0s - los

...

Epoch 2/50
23/23 - 0s - loss: 4789.1157 - 40ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 4178.3823 - 36ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 3580.9441 - 78ms/epoch - 3ms/step
Epoch 5/50
23/23 - 0s - loss: 3172.9243 - 45ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 2858.9951 - 43ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 2585.1382 - 40ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 2361.7354 - 37ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 2160.7454 - 42ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1983.2648 - 37ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 1822.4633 - 39ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 1678.2644 - 41ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 1545.8683 - 44ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 1423.0760 - 52ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 1312.7560 - 48ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 1213.0748 - 45ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 11

Epoch 29/50
23/23 - 0s - loss: 116.0518 - 40ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 115.8259 - 42ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 114.2687 - 42ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 113.3088 - 41ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 113.2918 - 42ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 114.2538 - 50ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 110.9835 - 43ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 111.0103 - 43ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 110.5860 - 40ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 112.4687 - 39ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 112.3179 - 41ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 108.1776 - 40ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 110.4699 - 46ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 107.2738 - 38ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 107.0922 - 41ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 107.4896 

Epoch 5/50
23/23 - 0s - loss: 5394.8120 - 45ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 4073.6748 - 45ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 3143.1899 - 44ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 2533.7910 - 44ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 2090.5967 - 42ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1761.0903 - 41ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 1491.2533 - 44ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 1280.5822 - 48ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 1109.3618 - 49ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 979.8696 - 53ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 866.3824 - 57ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 771.5892 - 51ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 690.2277 - 56ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 631.4807 - 54ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 568.8536 - 60ms/epoch - 3ms/step
Epoch 20/50
23/23 - 0s - loss: 515.1

Epoch 32/50
23/23 - 0s - loss: 720.1624 - 45ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 696.9585 - 43ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 674.3787 - 41ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 653.0747 - 38ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 632.0240 - 38ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 611.8147 - 37ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 593.3276 - 38ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 575.5089 - 39ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 557.8873 - 38ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 539.9693 - 44ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 523.6091 - 40ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 508.3747 - 40ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 492.4440 - 43ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 477.7758 - 45ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 463.8719 - 43ms/epoch - 2ms/step
Epoch 47/50
23/23 - 0s - loss: 449.8102 

Epoch 8/50
23/23 - 0s - loss: 205.3244 - 49ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 191.5741 - 50ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 179.0721 - 46ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 175.1506 - 50ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 169.3358 - 46ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 164.7958 - 50ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 161.0337 - 45ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 157.4640 - 41ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 154.6458 - 42ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 153.2865 - 42ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 150.3119 - 47ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 148.1593 - 52ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 147.2788 - 43ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 146.0412 - 47ms/epoch - 2ms/step
Epoch 22/50
23/23 - 0s - loss: 143.9522 - 44ms/epoch - 2ms/step
Epoch 23/50
23/23 - 0s - loss: 143.1963 - 

In [48]:
print(mse_listA)

[704.4635340142823, 251.6916907456877, 534.4151416312624, 371.1395793639569, 141.08737967084545, 127.05273957546127, 186.26893233359422, 268.7354015303435, 420.1041844195306, 93.657437518098, 120.64634557346464, 75.89332047157572, 231.85778098094883, 622.4368662412165, 132.20923251728368, 165.63014209568715, 648.8672943133, 507.4371764570743, 122.7454030004688, 136.91636833881435, 378.1833034524727, 1067.685956862128, 229.3244048340154, 245.73733277721544, 97.37378618136808, 139.12499526575712, 198.83942935120373, 136.7232227217198, 101.7271611124768, 129.47210732077167, 390.2823619300157, 1294.5068330850706, 281.450632742395, 199.38431011213822, 111.13358937801017, 266.75338818077626, 139.71201636939568, 189.79727402184028, 137.50166653756557, 164.64805960976042, 116.98316868681297, 436.69136363965333, 373.2165013752355, 134.82345499487624, 176.6766068867306, 356.87963330425504, 114.19786274835042, 1817.880522322054, 118.88587372556508, 182.422132344173]


In [49]:
df_results = pd.DataFrame.from_dict({"Part": ["A"], "mean mse": [np.mean(mse_listA)], "std mse": [np.std(mse_listA)]})
df_results

,Part,mean mse,std mse
0,A,311.825498,322.365672
